## Similarity based Recommendation System: Beers

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
! curl -O https://s3.amazonaws.com/demo-datasets/beer_reviews.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 27.3M  100 27.3M    0     0  2021k      0  0:00:13  0:00:13 --:--:-- 2206k


In [3]:
df = pd.read_csv("beer_reviews.tar.gz", compression='gzip')


/Users/raymondma/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [4]:
df.head()

,beer_reviews/,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1586629 entries, 0 to 1586628
Data columns (total 13 columns):
beer_reviews/         1586629 non-null object
brewery_name          1586611 non-null object
review_time           1586614 non-null float64
review_overall        1586614 non-null float64
review_aroma          1586614 non-null float64
review_appearance     1586614 non-null float64
review_profilename    1586266 non-null object
beer_style            1586614 non-null object
review_palate         1586614 non-null float64
review_taste          1586614 non-null float64
beer_name             1586614 non-null object
beer_abv              1518829 non-null float64
beer_beerid           1586614 non-null float64
dtypes: float64(8), object(5)
memory usage: 169.5+ MB


In [6]:
n = 250
top_n = df.beer_name.value_counts().index[:n]
df = df[df.beer_name.isin(top_n)]
df.head()

,beer_reviews/,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
798,1075,Caldera Brewing Company,1212201268,4.5,4.5,4,grumpy,American Double / Imperial Stout,4.0,4.5,Imperial Stout,NaN,42964
1559,11715,Destiny Brewing Company,1137124057,4.0,3.5,4,blitheringidiot,American Pale Ale (APA),3.5,3.5,Pale Ale,4.5,26420
1560,11715,Destiny Brewing Company,1129504403,4.0,2.5,4,NeroFiddled,American Pale Ale (APA),4.0,3.5,Pale Ale,4.5,26420
1563,11715,Destiny Brewing Company,1137125989,3.5,3.0,4,blitheringidiot,American IPA,4.0,4.0,IPA,NaN,26132
1564,11715,Destiny Brewing Company,1130936611,3.0,3.0,3,Gavage,American IPA,4.0,3.5,IPA,NaN,26132


In [7]:
df_pivot = pd.pivot_table(df, values=["review_overall"],
        columns=["beer_name", "review_profilename"],
        aggfunc=np.mean)
df_wide = df_pivot.unstack(-1)
df_wide.head()

review_profilename                         0110x011  02maxima  03SVTCobra  \
               beer_name                                                    
review_overall #9                               NaN       NaN         NaN   
               120 Minute IPA                   NaN       NaN         NaN   
               1554 Enlightened Black Ale       NaN       NaN         NaN   
               60 Minute IPA                    NaN       NaN         NaN   
               90 Minute IPA                      5       NaN         NaN   

review_profilename                         05Harley  0Naught0  0beerguy0  \
               beer_name                                                   
review_overall #9                               NaN       NaN        NaN   
               120 Minute IPA                     4       NaN        NaN   
               1554 Enlightened Black Ale       NaN       NaN        NaN   
               60 Minute IPA                    NaN       NaN        NaN   
               90 Minute IPA                      4       NaN        NaN   

review_profilename                         0runkp0s  0tt0  1000Bottles  \
               beer_name                                                 
review_overall #9                               NaN   NaN          NaN   
               120 Minute IPA                   NaN   1.5          NaN   
               1554 Enlightened Black Ale       NaN   NaN          NaN   
               60 Minute IPA                    NaN   NaN          NaN   
               90 Minute IPA                    NaN   NaN          NaN   

review_profilename                         1001111.0   ...    zuker  \
               beer_name                               ...            
review_overall #9                                NaN   ...      NaN   
               120 Minute IPA                    NaN   ...      NaN   
               1554 Enlightened Black Ale        NaN   ...      NaN   
               60 Minute IPA                     NaN   ...      NaN   
               90 Minute IPA                     NaN   ...      NaN   

review_profilename                         zulufactor  zumicroom  zwalk8  \
               beer_name                                                   
review_overall #9                                 NaN        NaN     NaN   
               120 Minute IPA                     NaN        NaN     NaN   
               1554 Enlightened Black Ale         NaN        NaN     NaN   
               60 Minute IPA                      NaN        NaN     NaN   
               90 Minute IPA                      NaN        NaN     NaN   

review_profilename                         zwoehr  zymrgy  zymurgy4all  \
               beer_name                                                 
review_overall #9                             NaN     NaN          NaN   
               120 Minute IPA                 NaN     NaN          NaN   
               1554 Enlightened Black Ale     NaN     NaN          NaN   
               60 Minute IPA                  NaN     NaN          NaN   
               90 Minute IPA                  NaN     NaN          NaN   

review_profilename                         zymurgywhiz  zythus  zyzygy  
               beer_name                                                
review_overall #9                                  NaN     NaN     NaN  
               120 Minute IPA                      NaN     NaN     NaN  
               1554 Enlightened Black Ale          NaN     NaN     NaN  
               60 Minute IPA                       NaN     NaN     NaN  
               90 Minute IPA                       NaN     NaN     NaN  

[5 rows x 22140 columns]

In [8]:
df_wide = df_wide.fillna(0)

In [9]:
df_wide.columns[:10]

Index([u'0110x011', u'02maxima', u'03SVTCobra', u'05Harley', u'0Naught0',
       u'0beerguy0', u'0runkp0s', u'0tt0', u'1000Bottles', u'1001111.0'],
      dtype='object', name=u'review_profilename')

In [10]:
dists = cosine_similarity(df_wide)
dists

array([[ 1.        ,  0.27540494,  0.27410345, ...,  0.32928048,
         0.34805798,  0.31249922],
       [ 0.27540494,  1.        ,  0.25151873, ...,  0.2854835 ,
         0.23301356,  0.2802485 ],
       [ 0.27410345,  0.25151873,  1.        , ...,  0.31629515,
         0.22521858,  0.2737628 ],
       ..., 
       [ 0.32928048,  0.2854835 ,  0.31629515, ...,  1.        ,
         0.28025764,  0.34504013],
       [ 0.34805798,  0.23301356,  0.22521858, ...,  0.28025764,
         1.        ,  0.25526913],
       [ 0.31249922,  0.2802485 ,  0.2737628 , ...,  0.34504013,
         0.25526913,  1.        ]])

In [14]:
dists = pd.DataFrame(dists)
dists.columns = beer_names
dists.index = beer_names
dists.ix[0:10, 0:10]

beer_name,#9,120 Minute IPA,1554 Enlightened Black Ale,60 Minute IPA,90 Minute IPA,Aecht Schlenkerla Rauchbier Märzen,AleSmith IPA,AleSmith Speedway Stout,Allagash White,Alpha King Pale Ale
beer_name,,,,,,,,,,
#9,1.000000,0.275405,0.274103,0.388364,0.365175,0.253841,0.228479,0.227612,0.340681,0.293315
120 Minute IPA,0.275405,1.000000,0.251519,0.378258,0.410366,0.262425,0.315971,0.337541,0.282273,0.336796
1554 Enlightened Black Ale,0.274103,0.251519,1.000000,0.319887,0.314028,0.252486,0.266866,0.261761,0.260275,0.307296
60 Minute IPA,0.388364,0.378258,0.319887,1.000000,0.533042,0.316928,0.312343,0.307627,0.360975,0.385249
90 Minute IPA,0.365175,0.410366,0.314028,0.533042,1.000000,0.312861,0.344218,0.358754,0.356804,0.418582
Aecht Schlenkerla Rauchbier Märzen,0.253841,0.262425,0.252486,0.316928,0.312861,1.000000,0.244490,0.246063,0.297672,0.263248
AleSmith IPA,0.228479,0.315971,0.266866,0.312343,0.344218,0.244490,1.000000,0.521889,0.277409,0.400741
AleSmith Speedway Stout,0.227612,0.337541,0.261761,0.307627,0.358754,0.246063,0.521889,1.000000,0.273930,0.420247
Allagash White,0.340681,0.282273,0.260275,0.360975,0.356804,0.297672,0.277409,0.273930,1.000000,0.295666


In [16]:
df_wide.index.levels[0]
beer_names = df_wide.index.levels[1]

Index([u'#9', u'120 Minute IPA', u'1554 Enlightened Black Ale',
       u'60 Minute IPA', u'90 Minute IPA',
       u'Aecht Schlenkerla Rauchbier Märzen', u'AleSmith IPA',
       u'AleSmith Speedway Stout', u'Allagash White', u'Alpha King Pale Ale', 
       ...
       u'Vanilla Porter', u'Weihenstephaner Hefeweissbier',
       u'Weihenstephaner Korbinian', u'Westmalle Trappist Dubbel',
       u'Westmalle Trappist Tripel', u'World Wide Stout',
       u'Yeti Imperial Stout', u'Young's Double Chocolate Stout',
       u'Yuengling Traditional Lager', u'Éphémère (Apple)'],
      dtype='object', name=u'beer_name', length=250)


In [17]:
beers_i_like = ['Sierra Nevada Pale Ale', '120 Minute IPA', 'Allagash White']
dists[beers_i_like].head()

beer_name,Sierra Nevada Pale Ale,120 Minute IPA,Allagash White
beer_name,,,
#9,0.373968,0.275405,0.340681
120 Minute IPA,0.301693,1.000000,0.282273
1554 Enlightened Black Ale,0.330033,0.251519,0.260275
60 Minute IPA,0.459641,0.378258,0.360975
90 Minute IPA,0.441189,0.410366,0.356804


In [18]:
beers_summed = dists[beers_i_like].apply(lambda row: np.sum(row), axis=1)

In [20]:
#ranking 
beers_summed = beers_summed.order(ascending=False)
beers_summed

beer_name
Sierra Nevada Pale Ale                        1.654205
Allagash White                                1.634784
120 Minute IPA                                1.583966
HopDevil Ale                                  1.224217
Sierra Nevada Celebration Ale                 1.215156
90 Minute IPA                                 1.208359
60 Minute IPA                                 1.198874
Stone Ruination IPA                           1.194210
Stone IPA (India Pale Ale)                    1.193193
Storm King Stout                              1.192405
Arrogant Bastard Ale                          1.189981
Sierra Nevada Bigfoot Barleywine Style Ale    1.178245
Prima Pils                                    1.178093
Brooklyn Black Chocolate Stout                1.156365
Ayinger Celebrator Doppelbock                 1.148356
Hennepin (Farmhouse Saison)                   1.147501
Samuel Adams Boston Lager                     1.146304
Hop Rod Rye                                   1.140271


In [21]:
#function to find simliar beers of input
def get_similar(beers, n=None):
    """
    calculates which beers are most similar to the beers provided. Does not return
    the beers that were provided
    
    Parameters
    ----------
    beers: list
        some beers!
    
    Returns
    -------
    ranked_beers: list
        rank ordered beers
    """
    beers = [beer for beer in beers if beer in dists.columns]
    beers_summed = dists[beers].apply(lambda row: np.sum(row), axis=1)
    beers_summed = beers_summed.order(ascending=False)
    ranked_beers = beers_summed.index[beers_summed.index.isin(beers)==False]
    ranked_beers = ranked_beers.tolist()
    if n is None:
        return ranked_beers
    else:
        return ranked_beers[:n]

In [22]:
for i, beer in enumerate(get_similar(["Coors Light", "Bud Light", "Amstel Light"], 10)):
    print "%d) %s" % (i+1, beer)

1) Miller Lite
2) Budweiser
3) Corona Extra
4) Samuel Adams Boston Lager
5) Heineken Lager Beer
6) Blue Moon Belgian White
7) Guinness Draught
8) Miller High Life
9) Samuel Adams Summer Ale
10) Sierra Nevada Pale Ale
